# Import libraries

In [1]:
is_going_to_train = 1

base_model_path = './model/fer2013_simple_128_out_testing_nontop_1.h5'
final_model_path = './model/fer2013_simple_128_out_testing_1.h5'

final_model_json_path = './model/fer2013_simple_128_out_testing_nontop_1.json'
base_model_json_path = './model/fer2013_simple_128_out_testing_1.json'

# train_path = 'E:/EmotiW2017/Train_AFEW/face_images'
# val_path = 'E:/EmotiW2017/Val_AFEW/face_images'

train_path = './data/fer2013/train'
val_path = './data/fer2013/test1'

val_path_afew = 'E:/EmotiW2017/Val_AFEW/face_images'
# val_path_afew = './data/fer2013/test1'
# val_path_afew = './data/fer2013/test2'

his_path = './model/fer2013_simple_128_out_testing_1.pickle'

In [2]:
from tensorflow.contrib.keras.python.keras import applications, layers
from tensorflow.contrib.keras.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.contrib.keras.python.keras import optimizers
from tensorflow.contrib.keras.python.keras.models import Sequential, Model
from tensorflow.contrib.keras.python.keras.layers import Dropout, Flatten, Dense, Input, ZeroPadding2D, Conv2D, BatchNormalization, Activation, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.contrib.keras.python.keras.callbacks import EarlyStopping
from tensorflow.contrib.keras.python.keras.optimizers import Adam, RMSprop

from tensorflow.contrib.keras.python.keras.applications.imagenet_utils import _obtain_input_shape
from tensorflow.contrib.keras.python.keras import backend as K

# CNN-LSTM
from tensorflow.contrib.keras.python.keras.layers import LSTM
from tensorflow.contrib.keras.python.keras.layers.embeddings import Embedding
from tensorflow.contrib.keras.python.keras.preprocessing import sequence
from tensorflow.contrib.keras.python.keras.layers.wrappers import TimeDistributed

from residual_funcs import *

import numpy as np
import matplotlib.pyplot as plt

# Visualization
from IPython.display import SVG
from tensorflow.contrib.keras.python.keras.utils.vis_utils import model_to_dot
from tensorflow.contrib.keras.python.keras.utils import plot_model

import pickle

# Some functions

In [3]:
def get_emotion(ohv):
    if ohv.shape[0] == 1:
        indx = ohv[0]
    else:
        indx = np.argmax(ohv)
        
    if indx == 0:
        return 'angry'
    elif indx == 1:
        return 'disgust'
    elif indx == 2:
        return 'fear'
    elif indx == 3:
        return 'happy'
    elif indx == 4:
        return 'sad'
    elif indx == 5:
        return 'surprise'
    elif indx == 6:
        return 'neutral'

# CNN

In [4]:
def CNN():
    base_model = Sequential()

    base_model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape = (48, 48, 1)))

    base_model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))

    base_model.add(MaxPooling2D(pool_size=(2, 2)))
    base_model.add(Dropout(0.2))

    base_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))

    base_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    
    base_model.add(MaxPooling2D(pool_size=(2, 2)))
    base_model.add(Dropout(0.2))

    base_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    
    base_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    
    base_model.add(MaxPooling2D(pool_size=(2, 2)))
    base_model.add(Dropout(0.2))

    base_model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    
    base_model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))

    base_model.add(MaxPooling2D(pool_size=(2, 2)))
    base_model.add(Dropout(0.2))
    
    base_model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    
    base_model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))

    base_model.add(MaxPooling2D(pool_size=(2, 2)))
    base_model.add(Dropout(0.2))

    base_model.add(Flatten())
    
    return base_model.output, base_model.input, base_model

# FC layer

In [5]:
if is_going_to_train == 1:
    # add a global spatial average pooling layer
    out, inp, base_model = CNN()

In [6]:
if is_going_to_train == 1:
    # let's add 2 fully-connected layers
    x = Dense(512, activation='relu')(out)
    x = Dropout(0.2)(x)
    # Softmax classifier
    predictions = Dense(7, activation='softmax')(x)

In [7]:
if is_going_to_train == 1:
    # this is the model we will train
    model = Model(inputs=inp, outputs=predictions)

# Review model architecture

In [8]:
if is_going_to_train == 1:
    print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 48, 48, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 128)       147584    
__________

# Compile model

In [ ]:
if is_going_to_train == 1:
    # compile the model (should be done *after* setting layers to non-trainable)
    opt = Adam(lr=0.0001)
    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit model

In [ ]:
if is_going_to_train == 1:
    early_stopping = EarlyStopping(monitor='val_loss', verbose=1, patience=50)

    # fer2013 ------------------------------------------------
    train_datagen = ImageDataGenerator()
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(48, 48),
        batch_size=128,
        class_mode='sparse',
        color_mode='grayscale')

    # classes=['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

    val_datagen = ImageDataGenerator()
    val_generator = val_datagen.flow_from_directory(
        val_path,
        target_size=(48, 48),
        batch_size=97,
        class_mode='sparse',
        color_mode='grayscale')

    history = model.fit_generator(generator=train_generator, steps_per_epoch=323,
                        epochs=200, verbose=1, callbacks=[early_stopping],
                        validation_data=val_generator, validation_steps=37)

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
Epoch 1/200
323/323 [==============================] - 226s - loss: 1.8442 - acc: 0.2428 - val_loss: 1.8353 - val_acc: 0.2494
Epoch 2/200
323/323 [==============================] - 25s - loss: 1.8101 - acc: 0.2487 - val_loss: 1.7976 - val_acc: 0.2809
Epoch 3/200
323/323 [==============================] - 25s - loss: 1.6670 - acc: 0.3379 - val_loss: 1.5550 - val_acc: 0.3976
Epoch 4/200
323/323 [==============================] - 25s - loss: 1.5078 - acc: 0.4169 - val_loss: 1.4512 - val_acc: 0.4383
Epoch 5/200
323/323 [==============================] - 25s - loss: 1.3858 - acc: 0.4686 - val_loss: 1.3014 - val_acc: 0.5110
Epoch 6/200
323/323 [==============================] - 25s - loss: 1.2950 - acc: 0.5070 - val_loss: 1.2354 - val_acc: 0.5325
Epoch 7/200
323/323 [==============================] - 25s - loss: 1.2256 - acc: 0.5356 - val_loss: 1.2001 - val_acc: 0.5450
Epoch 8/200
323/323 [==================

# Save model

In [ ]:
if is_going_to_train == 1:
    jsonstring = model.to_json()
    with open(final_model_json_path,'w') as f:
       f.write(jsonstring)
    model.save_weights(final_model_path, overwrite=True)
    
    jsonstring = base_model.to_json()
    with open(base_model_json_path,'w') as f:
       f.write(jsonstring)
    base_model.save_weights(base_model_path, overwrite=True)

# Load model & review loaded architecture

In [ ]:
if is_going_to_train == 0:
    with open(final_model_json_path, 'r') as f:
        json_string = f.read()
    model = model_from_json(json_string)
    model.load_weights(final_model_path)

    with open(base_model_json_path, 'r') as f:
        json_string = f.read()
    base_model = model_from_json(json_string)
    base_model.load_weights(base_model_path)

    print(model.summary())

# Evaluation

In [ ]:
# AFEW17 ------------------------------------------------
val_datagen_afew = ImageDataGenerator()
val_generator_afew = val_datagen_afew.flow_from_directory(
    val_path_afew,
    target_size=(48, 48),
    batch_size=1,
    class_mode='sparse',
    color_mode='grayscale')

# model.evaluate_generator(generator=val_generator_afew, steps=372)
model.evaluate_generator(generator=val_generator_afew, steps=3589)

In [ ]:
# SVG(model_to_dot(model).create(prog='dot', format='svg'))
# plot_model(model, to_file='model.png')

# Training history

In [ ]:
if is_going_to_train == 1:
    with open(his_path, 'wb') as f:
        pickle.dump(history.history, f)
    history_ = history.history
else:
    file = open(his_path,'rb')
    history_ = pickle.load(file)

In [ ]:
# list all data in history
print(history_.keys())
# summarize history for accuracy
plt.plot(history_['acc'])
plt.plot(history_['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history_['loss'])
plt.plot(history_['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()